In [1]:
# conda config --add channels conda-forge
# conda install numpy
# conda install pandas openpyxl

In [2]:
import os
import glob
import re
import openpyxl
# these packages were installed on the command line using conda install numpy
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
from pathlib import Path

In [3]:
# changing working directory
os.chdir("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/PHE Database/2. Coding/2.2. Data Extraction/Input")

In [4]:
# reading in files
# file path of all the ASC files
#folder_path = Path("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/PHE Database/2. Coding/2.2. Data Extraction/Input")

# How many asc files are in the folder?
#for filename in glob.glob(os.path.join(folder_path,'*','*.asc')):
#asc_files = list(glob.glob("**/*.asc", recursive = True))
#print(f"Found {len(asc_files)} ASC files.")

In [5]:
path = "/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/PHE Database/2. Coding/2.2. Data Extraction/Input"
folder_names = os.listdir(path)

def get_folder_path(names):
    # creating a function to get the paths for each folder
    count = 0
    path_array = []
    for i in names:
        folder_path = path + "/" + names[count]
        path_array.append(folder_path)

        count += 1

    return path_array

In [6]:
# calling fn
ohio_tmax_2020_path = get_folder_path(folder_names)[0]
ohio_tmax_2021_path = get_folder_path(folder_names)[1]
ohio_tmax_2022_path = get_folder_path(folder_names)[9]
ohio_HI_2020_path = get_folder_path(folder_names)[8]
ohio_HI_2021_path = get_folder_path(folder_names)[10]
ohio_HI_2022_path = get_folder_path(folder_names)[2]
ohio_tmin_2020_path = get_folder_path(folder_names)[7]
ohio_tmin_2021_path = get_folder_path(folder_names)[5]
ohio_tmin_2022_path = get_folder_path(folder_names)[12]
ohio_PM_2020_path = get_folder_path(folder_names)[6]
ohio_PM_2021_path = get_folder_path(folder_names)[4]
ohio_PM_2022_path = get_folder_path(folder_names)[11]

In [7]:
# in python, we need count to remember the index
# the iterator, 'i', gives us the actual value for that index
def getdata(path_name, doy_index):
    count = 0
    all_df = pd.DataFrame()

    # getting names of all the files
    asc_files = glob.glob(path_name + "**/*.asc")
    
    for i in asc_files:
        # name of the parent folder
        folder_name = os.path.dirname(asc_files[count])

        # name of the file
        file_stem = os.path.basename(asc_files[count])

        # splitting the string of file stem to get year, DOY, state, and the variable
        # split by an underscore or period
        split_str = re.split(r'[_.]+', file_stem) 
        doy = split_str[doy_index]

        with rasterio.open(i) as src:
        
            data = src.read(1)
            transform = src.transform
            nodata = src.nodata
            rows, cols = data.shape
            row_inds, col_inds = np.meshgrid(np.arange(rows), np.arange(cols), indexing = 'ij')
            xs, ys = rasterio.transform.xy(transform, row_inds, col_inds)
    
            flat_df = pd.DataFrame({
                'DOY': doy, 
                'x': np.array(xs).flatten(),
                'y': np.array(ys).flatten(),
                'Value': data.flatten()
            })
    
            if nodata is not None:
                flat_df = flat_df[flat_df['Value'] != nodata]

                # storing data
                all_df = pd.concat([all_df, flat_df])
                
        count += 1
        
    return(all_df)

In [8]:
# calling fn
ohio_tmax_2020_df = getdata(ohio_tmax_2020_path, -4)
ohio_tmax_2021_df = getdata(ohio_tmax_2021_path, -4)
ohio_tmax_2022_df = getdata(ohio_tmax_2022_path, -4)
ohio_tmin_2020_df = getdata(ohio_tmin_2020_path, -4)
ohio_tmin_2021_df = getdata(ohio_tmin_2021_path, -4)
ohio_tmin_2022_df = getdata(ohio_tmin_2022_path, -4)
ohio_HI_2020_df = getdata(ohio_HI_2020_path, -2)
ohio_HI_2021_df = getdata(ohio_HI_2021_path, -2)
ohio_HI_2022_df = getdata(ohio_HI_2022_path, -2)
ohio_PM_2020_df = getdata(ohio_PM_2020_path, -2)
ohio_PM_2021_df = getdata(ohio_PM_2021_path, -2)
ohio_PM_2022_df = getdata(ohio_PM_2022_path, -2)

In [9]:
# viewing data
ohio_tmax_2020_df.head()

,DOY,x,y,Value
791,50,2.032326e+06,4.815818e+06,76.099998
792,50,2.033252e+06,4.815818e+06,76.099998
1204,50,2.030473e+06,4.814892e+06,75.900002
1205,50,2.031399e+06,4.814892e+06,76.900002
1206,50,2.032326e+06,4.814892e+06,76.500000


In [10]:
# # creating 1 df
# final_df = pd.concat([ohio_tmax_2020_df, ohio_tmax_2021_df])
# final_df.head()

In [26]:
# Getting number of dimensions
ohio_tmax_2020_df.shape[0] 

20037710

In [29]:
# exporting data
output = "/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/PHE Database/2. Coding/2.2. Data Extraction/Output"
ohio_tmax_2020_df.to_csv(output + "/" + 'Ohio_Tmax_2020.csv')
ohio_tmax_2021_df.to_csv(output + "/" + 'Ohio_Tmax_2021.csv')
ohio_tmax_2022_df.to_csv(output + "/" + 'Ohio_Tmax_2022.csv')
ohio_PM_2020_df.to_csv(output + "/" + 'Ohio_PM_2020.csv')
ohio_PM_2021_df.to_csv(output + "/" + 'Ohio_PM_2021.csv')
ohio_PM_2022_df.to_csv(output + "/" + 'Ohio_PM_2022.csv')
ohio_tmin_2020_df.to_csv(output + "/" + 'Ohio_Tmin_2020.csv')
ohio_tmin_2021_df.to_csv(output + "/" + 'Ohio_Tmin_2021.csv')
ohio_tmin_2022_df.to_csv(output + "/" + 'Ohio_Tmin_2022.csv')
ohio_HI_2020_df.to_csv(output + "/" + 'Ohio_HI_2020.csv')
ohio_HI_2021_df.to_csv(output + "/" + 'Ohio_HI_2021.csv')
ohio_HI_2022_df.to_csv(output + "/" + 'Ohio_HI_2022.csv')